In [22]:
import tensorflow as tf
from transformers import BertConfig, AutoTokenizer, TFBertModel, BertTokenizer, TFBertForSequenceClassification, BertModel

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss

import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.utils import pad_sequences

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

from keras.layers import Embedding


In [23]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=6)
model.build()
model.load_weights("../data/bert_classif_VM.keras")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/irismarechal/.pyenv/versions/3.10.6/envs/moder_ia/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [24]:
def hamming_loss(y_true, y_pred, threshold=0.5):
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

# Apply the threshold to the predicted values
    y_pred = tf.cast(y_pred > threshold, dtype=tf.float32)

 # Calculate the Hamming Loss\
    epsilon = 1e-7  # Small epsilon value\n",
    hamming_loss = 1.0 - tf.reduce_mean(tf.reduce_sum(y_true * y_pred, axis=1) / (tf.reduce_sum(y_true + y_pred - y_true * y_pred, axis=1) + epsilon))

    return hamming_loss.numpy()

In [25]:
data_extract = pd.read_csv("../data/"+"labelling_dataset_v1.csv")
data_extracted=data_extract[0:1000]

#process X et Y
texts = data_extracted['text_processed']
labels = data_extracted.drop(['text','text_processed'],axis=1)

#transfor the X into a list of string
text_to_encode=texts.values.tolist()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

X_tokenized=tokenizer(text_to_encode, max_length=200, padding='max_length', return_attention_mask=True, return_tensors='tf')

X_tuple = (X_tokenized['input_ids'], X_tokenized['token_type_ids'], X_tokenized['attention_mask'])


#Y pred + apply function
y_pred = model.predict(X_tuple)
y_true = labels

32/32 [==============================] - 214s 7s/step


In [27]:
h3 = hamming_loss(y_true, y_pred)
h3

ValueError: Attempt to convert a value (TFSequenceClassifierOutput(loss=None, logits=array([[-0.64942425, -0.35383356, -0.50275403, -0.11968189, -0.2865996 ,
        -0.42911953],
       [-0.6881017 , -0.53711444, -0.5405972 , -0.23513767,  0.18102808,
        -0.47056854],
       [-0.37870422, -0.33187675, -0.40674347, -0.25147885, -0.39447534,
        -0.5836401 ],
       ...,
       [-0.43493688, -0.20578143, -0.32088625, -0.3356344 , -0.43986544,
        -0.44403508],
       [-0.56450665, -0.29238895, -0.43692702, -0.1734829 , -0.4659426 ,
        -0.5295357 ],
       [-0.9298853 , -0.6529261 , -0.6143728 ,  0.38198948,  0.36555964,
        -0.24915746]], dtype=float32), hidden_states=None, attentions=None)) with an unsupported type (<class 'transformers.modeling_tf_outputs.TFSequenceClassifierOutput'>) to a Tensor.

In [28]:
logits = y_pred.logits
probabilities = tf.nn.softmax(logits, axis=1)
predicted_classes = tf.argmax(probabilities, axis=1)

# Create a dictionary containing the data
data = {
    "Text": texts,
    "Predicted_Class": predicted_classes,
}

# Add probability columns for each class
num_classes = model.config.num_labels
for class_idx in range(num_classes):
    data[f"Probability_Class_{class_idx}"] = probabilities[:, class_idx]

df = pd.DataFrame(data)

In [30]:
df = df[['Probability_Class_0', 'Probability_Class_1',
       'Probability_Class_2', 'Probability_Class_3', 'Probability_Class_4',
       'Probability_Class_5']]

In [33]:
h3 = hamming_loss(df, y_true)
h3

0.9070498

In [ ]:
y_pred = df.to_numpy()
y_pred

In [ ]:
y_true

In [ ]:
y_true.columns

In [ ]:
y_pred_df=pd.DataFrame(y_pred)


In [ ]:
y_pred_df.sum()

In [ ]:
h3 = hamming_loss(y_true, y_pred_df)
h3

In [34]:
tweet_1="black people should die"

In [35]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tweet_tokenized=tokenizer(tweet_1, max_length=200, padding='max_length', return_attention_mask=True, return_tensors='tf')
tweet_tuple = (tweet_tokenized['input_ids'], tweet_tokenized['token_type_ids'], tweet_tokenized['attention_mask'])

In [36]:
y_pred_tweet = model.predict(tweet_tuple)
y_pred_tweet

1/1 [==============================] - 1s 503ms/step


TFSequenceClassifierOutput(loss=None, logits=array([[-0.9068673 , -0.6819405 , -0.66433483,  0.42534655,  0.62063384,
        -0.19836582]], dtype=float32), hidden_states=None, attentions=None)

In [37]:
probabilities = tf.nn.softmax(y_pred_tweet.logits, axis=-1).numpy()
probabilities

array([[0.07166547, 0.08974171, 0.09133568, 0.27157074, 0.33013776,
        0.14554864]], dtype=float32)

In [38]:
predicted_classes = tf.argmax(probabilities, axis=1)

In [41]:
data = {
    "Text": tweet_1,
    "Predicted_Class": predicted_classes,
}

# Add probability columns for each class
num_classes = model.config.num_labels
for class_idx in range(num_classes):
    data[f"Probability_Class_{class_idx}"] = probabilities[:, class_idx]

df_1 = pd.DataFrame(data)

df_1

,Text,Predicted_Class,Probability_Class_0,Probability_Class_1,Probability_Class_2,Probability_Class_3,Probability_Class_4,Probability_Class_5
0,black people should die,4,0.071665,0.089742,0.091336,0.271571,0.330138,0.145549


In [49]:
y_response=df_1['Predicted_Class']

In [51]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Text                 1 non-null      object 
 1   Predicted_Class      1 non-null      int64  
 2   Probability_Class_0  1 non-null      float32
 3   Probability_Class_1  1 non-null      float32
 4   Probability_Class_2  1 non-null      float32
 5   Probability_Class_3  1 non-null      float32
 6   Probability_Class_4  1 non-null      float32
 7   Probability_Class_5  1 non-null      float32
dtypes: float32(6), int64(1), object(1)
memory usage: 168.0+ bytes


In [52]:
#mapping_dict = {'Probability_Class_0': 'Racist tweet', 'Probability_Class_1': 'Religious Hate', 'Probability_Class_2': 'Xenophobia','Probability_Class_3': 'Misogyny','Probability_Class_4': 'Transphobia','Probability_Class_5': 'Homophobia'}

mapping_dict = {0: 'Racist tweet', 1: 'Religious Hate', 2: 'Xenophobia',3: 'Misogyny',4: 'Transphobia',5: 'Homophobia'}

y_response=df_1['Predicted_Class'].map(mapping_dict)
y_response

0    Transphobia
Name: Predicted_Class, dtype: object

In [ ]:
config = BertConfig.from_pretrained('prajjwal1/bert-mini')
model = TFBertForSequenceClassification(config=config)


In [ ]:
model.build()

In [ ]:
model.load_weights("weights-bert.h5")

In [ ]:
model.summary()

In [14]:
def hamming_loss(y_true, y_pred, threshold=0.5):
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

# Apply the threshold to the predicted values
    y_pred = tf.cast(y_pred > threshold, dtype=tf.float32)

 # Calculate the Hamming Loss\
    epsilon = 1e-7  # Small epsilon value\n",
    hamming_loss = 1.0 - tf.reduce_mean(tf.reduce_sum(y_true * y_pred, axis=1) / (tf.reduce_sum(y_true + y_pred - y_true * y_pred, axis=1) + epsilon))

    return hamming_loss.numpy()

In [15]:
data_processed = pd.read_csv("../data/"+"labelling_dataset_v1.csv")
# Split into training and testing data
X = data_processed["text_processed"]
y = data_processed.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

# Size of your embedding space = size of the vector representing each word
embedding_size = 50

# create the model
model = Sequential()

model.add(Embedding(input_dim=vocab_size+1, output_dim=embedding_size))
model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.LSTM(100))
model.add(layers.Dense(6, activation='sigmoid'))

# Students will be ending their code here

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Change the number of epochs and the batch size depending on the RAM Size

es = EarlyStopping(patience=4, restore_best_weights=True)

model.fit(X_train_pad, y_train,
    epochs=15,
    batch_size=128,
    callbacks=[es], validation_split=0.3
)
res = model.evaluate(X_test_pad, y_test)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')
print('Testing loss \t', res[0]*100)
print('Testing accuracy ', res[1]*100)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1958700   
                                                                 
 conv1d (Conv1D)             (None, None, 32)          4832      
                                                                 
 max_pooling1d (MaxPooling1  (None, None, 32)          0         
 D)                                                              
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 6)                 606       
                                                                 
Total params: 2017338 (7.70 MB)
Trainable params: 2017338 (7.70 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

KeyboardInterrupt: 

In [ ]:
y_true = y_test
y_pred=model.predict(X_test_pad)

h3 = hamming_loss(y_true, y_pred)
h3

In [17]:
data_processed = pd.read_csv("../data/"+"labelling_dataset_v1.csv")
# Split into training and testing data
X = data_processed["text_processed"]
y = data_processed.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

# Size of your embedding space = size of the vector representing each word
embedding_size = 50


# Create the model
model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1, # size of the input, impacting the number of weights in the linear combinations of the neurons of the first layer
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer
))

model.add(layers.GRU(20, return_sequences=True, activation="tanh"))
model.add(layers.GRU(20, activation="tanh"))
model.add(layers.Dense(6, activation="sigmoid"))
model.summary()

model.compile(loss='binary_crossentropy',
            optimizer='rmsprop',
            metrics=['accuracy'])

es = EarlyStopping(patience=4, restore_best_weights=True)

model.fit(X_train_pad, y_train,
        epochs=3,
        batch_size=128,
        callbacks=[es], validation_split=0.3
        )

res = model.evaluate(X_test_pad, y_test)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')
print('Testing loss \t', res[0]*100)
print('Testing accuracy ', res[1]*100)



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 50)          1958700   
                                                                 
 gru_2 (GRU)                 (None, None, 20)          4320      
                                                                 
 gru_3 (GRU)                 (None, 20)                2520      
                                                                 
 dense_2 (Dense)             (None, 6)                 126       
                                                                 
Total params: 1965666 (7.50 MB)
Trainable params: 1965666 (7.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/3
232/232 [==============================] - 28s 86ms/step - loss: 0.5344 - accuracy: 0.3064 - val_loss: 0.5160 - val_accuracy: 0.3490
Epoch

In [18]:
y_true = y_test
y_pred=model.predict(X_test_pad)

h3 = hamming_loss(y_true, y_pred)
h3

442/442 [==============================] - 7s 11ms/step


0.8290219

In [21]:
y_pred.sum()

18742.92

In [ ]:
from PIL import Image
from IPython.display import display

# Load the image from a file
image = Image.open("../data/Background_site.png")  # Replace with the actual path to your image file

# Display the image
display(image)